In [1]:
pip install open_clip_torch datasets torch torchvision scikit-learn tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset  

dataset = load_dataset("ravisri/bird-presence-classification")  
train_ds = dataset["train"]  
test_ds = dataset["test"]  

# Your class mapping (from the Dataset Viewer)  
class_names = ["bird", "no_bird"]

README.md:   0%|          | 0.00/563 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/48.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2299 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/575 [00:00<?, ? examples/s]

In [3]:
from torchvision import transforms  
from torch.utils.data import Dataset, DataLoader  

preprocess = transforms.Compose([  
    transforms.CenterCrop(224),  # Models trained on 224x224, but images are 256x256  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],  
                         std=[0.26862954, 0.26130258, 0.27577711])  
])  

class HFDataset(Dataset):  
    def __init__(self, hf_ds, transform):  
        self.ds = hf_ds  
        self.transform = transform  

    def __len__(self):  
        return len(self.ds)  

    def __getitem__(self, idx):  
        row = self.ds[idx]  
        img = row["image"]  
        label = row["label"] # Already integer (0:bird, 1:no_bird)  
        return self.transform(img), label  

train_dataset = HFDataset(train_ds, preprocess)  
test_dataset  = HFDataset(test_ds, preprocess)  

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)  
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [4]:
import open_clip  
import torch  

device = "cuda" if torch.cuda.is_available() else "cpu"  
model, _, _ = open_clip.create_model_and_transforms(  
    'ViT-B-32', pretrained='laion2b_s34b_b79k'  
)  
tokenizer = open_clip.get_tokenizer("ViT-B-32")  
model = model.to(device).eval()  
FEATURE_DIM = model.visual.output_dim

open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [5]:
prompts = [  
    "There is a bird present in the picture.",      # label 0, "bird"  
    "There is no bird in the picture."              # label 1, "no_bird"  
]

In [23]:
from tqdm import tqdm  
from sklearn.metrics import classification_report  

def evaluate_clip_base(model, tokenizer, loader, prompts, class_names, device='cuda'):  
    model.eval()  
    text_tokens = tokenizer(prompts).to(device)  
    text_features = model.encode_text(text_tokens)  
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)  
    y_true, y_pred = [], []  
    with torch.no_grad():  
        for images, labels in tqdm(loader, desc="Zero-shot CLIP"):  
            images = images.to(device)  
            image_features = model.encode_image(images)  
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)  
            logits = 100. * image_features @ text_features.T  
            preds = logits.argmax(dim=1).cpu().numpy()  
            y_pred.extend(preds)  
            y_true.extend(labels.numpy())  
    print("\n=== ZERO-SHOT CLIP ===")  
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))  

evaluate_clip_base(model, tokenizer, test_loader, prompts, class_names, device)

Zero-shot CLIP: 100%|██████████| 18/18 [00:01<00:00,  9.36it/s]


=== ZERO-SHOT CLIP ===
              precision    recall  f1-score   support

        bird     0.8056    0.1007    0.1790       288
     no_bird     0.5195    0.9756    0.6780       287

    accuracy                         0.5374       575
   macro avg     0.6625    0.5382    0.4285       575
weighted avg     0.6628    0.5374    0.4281       575



In [30]:
import torch.nn as nn  
import torch.optim as optim  

class CLIPWithHead(nn.Module):  
    def __init__(self, base_model, feature_dim, num_classes):  
        super().__init__()  
        self.base_model = base_model  
        self.head = nn.Linear(feature_dim, num_classes)  
    def forward(self, images):  
        with torch.no_grad():  
            features = self.base_model.encode_image(images)  
            features = features / features.norm(dim=-1, keepdim=True)  
        return self.head(features)  

probe = CLIPWithHead(model, FEATURE_DIM, num_classes=2).to(device)  

def train_linear_head(probe, train_loader, num_epochs=5, lr=1e-3, device='cuda'):  
    optimizer = optim.Adam(probe.head.parameters(), lr=lr)  
    criterion = nn.CrossEntropyLoss()  
    probe.train()  
    for epoch in range(num_epochs):  
        running_loss = 0.  
        for images, labels in tqdm(train_loader, desc=f'Fine-tuning (Epoch {epoch+1})'):  
            images, labels = images.to(device), labels.to(device)  
            optimizer.zero_grad()  
            logits = probe(images)  
            loss = criterion(logits, labels)  
            loss.backward()  
            optimizer.step()  
            running_loss += loss.item()  
        print(f'Epoch {epoch+1} loss: {running_loss/len(train_loader):.4f}')  
    probe.eval()  

train_linear_head(probe, train_loader, num_epochs=100, lr=1e-3, device=device)

Fine-tuning (Epoch 1): 100%|██████████| 72/72 [00:07<00:00, 10.12it/s]


Epoch 1 loss: 0.6353


Fine-tuning (Epoch 2): 100%|██████████| 72/72 [00:07<00:00, 10.03it/s]


Epoch 2 loss: 0.5409


Fine-tuning (Epoch 3): 100%|██████████| 72/72 [00:07<00:00,  9.87it/s]


...

Epoch 98 loss: 0.2168


Fine-tuning (Epoch 99): 100%|██████████| 72/72 [00:07<00:00, 10.16it/s]


Epoch 99 loss: 0.2163


Fine-tuning (Epoch 100): 100%|██████████| 72/72 [00:07<00:00, 10.09it/s]

Epoch 100 loss: 0.2163


In [31]:
torch.save(probe.state_dict(), "clip_bird_probe.pt")  
# To reload:  
probe.load_state_dict(torch.load("clip_bird_probe.pt"))  
probe.eval()

/tmp/ipykernel_85/3042128908.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  probe.load_state_dict(torch.load("clip_bird_probe.pt"))


CLIPWithHead(
  (base_model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (patch_dropout): Identity()
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): ModuleList(
          (0-11): 12 x ResidualAttentionBlock(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (ls_1): Identity()
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=768, out_features=3072, bias=True)
              (gelu): GELU(approximate='none')
              (c_proj): Linear(in_features=3072, out_features=768, bias=True)
            )
            (ls_2): Identity()
          )
      

In [32]:
def evaluate_clip_finetuned(probe, loader, class_names, device='cuda'):  
    probe.eval()  
    y_true, y_pred = [], []  
    with torch.no_grad():  
        for images, labels in tqdm(loader, desc="Fine-tuned CLIP"):  
            images = images.to(device)  
            logits = probe(images)  
            preds = logits.argmax(dim=1).cpu().numpy()  
            y_pred.extend(preds)  
            y_true.extend(labels.numpy())  
    print("\n=== FINE-TUNED (Linear Head) ===")  
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))  

evaluate_clip_finetuned(probe, test_loader, class_names, device)

Fine-tuned CLIP: 100%|██████████| 18/18 [00:01<00:00,  9.28it/s]


=== FINE-TUNED (Linear Head) ===
              precision    recall  f1-score   support

        bird     0.9173    0.8472    0.8809       288
     no_bird     0.8576    0.9233    0.8893       287

    accuracy                         0.8852       575
   macro avg     0.8874    0.8853    0.8851       575
weighted avg     0.8875    0.8852    0.8851       575



In [33]:
from huggingface_hub import HfApi, HfFolder, upload_folder, create_repo  

# Log in (enter your token)  
from huggingface_hub import login  
login()

In [34]:
repo_id = "ravisri/clip-bird-detector"  
api = HfApi()  
api.create_repo(repo_id=repo_id, exist_ok=True)

RepoUrl('https://huggingface.co/ravisri/clip-bird-detector', endpoint='https://huggingface.co', repo_type='model', repo_id='ravisri/clip-bird-detector')

In [37]:
from huggingface_hub import HfApi  

api = HfApi()  
api.upload_file(  
    path_or_fileobj="clip_bird_probe.pt",    # Your local file  
    path_in_repo="clip_bird_probe.pt",       # The filename to use in the repo (can be the same)  
    repo_id=repo_id,  
    repo_type="model"  
)

clip_bird_probe.pt:   0%|          | 0.00/605M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ravisri/clip-bird-detector/commit/e9b164b6ca2ee4de36a879b099ca3b3b6eaf2a48', commit_message='Upload clip_bird_probe.pt with huggingface_hub', commit_description='', oid='e9b164b6ca2ee4de36a879b099ca3b3b6eaf2a48', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ravisri/clip-bird-detector', endpoint='https://huggingface.co', repo_type='model', repo_id='ravisri/clip-bird-detector'), pr_revision=None, pr_num=None)